In [4]:
import pandas as pd
import unicodedata
import re
import nltk

In [5]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import nltk

# Loading data via google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_df = pd.read_csv(r"/content/drive/My Drive/data/chrous-verse-dataset-metadata.csv")
data_df = pd.DataFrame(data_df)
data_df

,song_id,song_name,Artist,Lyrics,Type
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේ\nඔබ දුරයි නෙතට ...,Calm
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේ\n\nබඹරු නටන හැන්දෑවේ…\nසැ...,Calm
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,\nසමනළ මුදුනේ සිරිපද සිඹ සිඹ\nඋදා ඉරක් පායයි\n...,Calm
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැව\nපමණි මට අද ශේෂ ...,Calm
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා...\nඋදා හිරු එළියේ...\n...,Calm
...,...,...,...,...,...
157,S 48,Dangakaaree Man,Indrani Wijebandara,දඟකාරී මං මද්දුම දෝණි\nඔබට අකීකරු නෑ අම්මේ\nදඟ...,Sad
158,S 49,Edaa Wage Adath,Somathilaka Jayamaha,එදා වාගේ අදත් පොද වැස්ස වැටෙනවා\nමල් පිපුණු ගහ...,Sad
159,S 50,Paluwa Dewanath Karala,Gunadasa Kapuge,පාළුව දෙවනත් කරලා\nනාඩන් උලලේනෝ\nකාටත් නපුරුදා...,Sad
160,S 51,Pawena Nil Walawe,W. D. Amaradeva,පාවෙනා නිල් වලාවේ\nනදී තලාවේ\nරිදී ජලාසේ\nඇගේ ...,Sad


In [7]:
def lineBreakReplace(text):
    return text.replace("\r\n", " ")

In [8]:
data_df['Lyrics'] =data_df.apply(lambda x: lineBreakReplace(x.Lyrics), axis=1)

In [9]:
def removeEscapSequences(text):
    return "".join(ch for ch in text if unicodedata.category(ch)[0]!="C")

In [10]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [11]:
def remove_digits(text):
    return ''.join([i for i in text if not i.isdigit()])

In [12]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [13]:
def removeHtmlTags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

In [14]:

data_df['Lyrics'] = data_df.apply(lambda x: removeHtmlTags(x.Lyrics), axis=1)

In [15]:
data_df.head()

,song_id,song_name,Artist,Lyrics,Type
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේඔබ දුරයි නෙතට මා...,Calm
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේබඹරු නටන හැන්දෑවේ…සැදෑ අඳු...,Calm
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,සමනළ මුදුනේ සිරිපද සිඹ සිඹඋදා ඉරක් පායයිසැනසිල...,Calm
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැවපමණි මට අද ශේෂ වු...,Calm
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා...උදා හිරු එළියේ...අළුත...,Calm


In [16]:
def removeSpecialCharacters(sentence,keep_apostrophes=True):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
        return filtered_sentence
    return sentence

In [17]:
def replace_full_stops(sentence):
    return sentence.replace("."," ")

In [18]:
data_df['Lyrics'] = data_df.apply(lambda x: removeSpecialCharacters(x.Lyrics), axis=1)
data_df['Lyrics'] = data_df.apply(lambda x: replace_full_stops(x.Lyrics), axis=1)

In [19]:
def remove_english_letters(sentence):
    non_english_sentence = re.sub(r'[a-zA-Z]', '', sentence)
    return non_english_sentence

In [20]:
data_df['Lyrics']  = data_df['Lyrics'] .apply(lambda x: remove_english_letters(x))

In [21]:
lines = list(open('/content/drive/My Drive/data/stop-words-sinhala.txt',encoding="utf8"))
sinhala_stop_words = [re.sub("\n","",x) for x in lines]

In [22]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
data_df['token_list']  = data_df['Lyrics'] .apply(lambda x: tokenize_text(x))

In [25]:
def remove_sinhala_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in sinhala_stop_words]
    return filtered_tokens

In [26]:

data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_sinhala_stop_words(x.token_list), axis=1)

In [27]:
def remove_two_letter_words(tokenized_words):
    # Use list comprehension to filter out words with length 2
    filtered_words = [word for word in tokenized_words if len(word) > 3]
    return filtered_words

In [28]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_two_letter_words(x.token_list_after_sw), axis=1)

In [29]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [30]:
list_of_words = ' '.join(data_df['pre_processed_text']).split()

In [31]:
# from collections import Counter
# import numpy as np
# most_common_words = [word for word, count in Counter(list_of_words).items() if count in np.arange(50,100)]
# unique_words = [word for word, count in Counter(list_of_words).items() if count in [1,2,3]]
# all_words_to_remove = most_common_words+unique_words;
# def remove_most_common_unique_words(tokens):
#     filtered_tokens = [token for token in tokens if token not in all_words_to_remove]
#     return filtered_tokens

In [32]:
#data_df['token_list_after_common_words'] = data_df.apply(lambda x: remove_most_common_unique_words(x.token_list_after_sw), axis=1)

In [33]:
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [34]:
# list_of_words = ' '.join(data_df['pre_processed_text']).split()
# Counter(list_of_words)

In [35]:
data_df['word_count'] = data_df['pre_processed_text'].str.split().str.len()
#data_df.head(40)

In [36]:
def total_unique_words(words):
    return nltk.FreqDist(words)

print("Total number of unique words",total_unique_words(list_of_words))

Total number of unique words <FreqDist with 2849 samples and 4355 outcomes>


In [37]:
def total_words(words):
    return(len(words))
print("Total number of words",total_words(list_of_words))

Total number of words 4355


In [38]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [39]:
#X= data_df['pre_processed_text']
data_df["final_data"] = "__label__"+data_df['Type'] + " " + data_df["pre_processed_text"]
#data_df
#y=label_encoder.fit_transform(data_df['Type'])

In [40]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [42]:
#model_checkpoint = "xlm-roberta-base"
# #tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
# model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

In [43]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [44]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [45]:
data_df['label'] =labelencoder.fit_transform(data_df['Type'])

In [46]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
def tokenize_function(examples):
    return tokenizer(examples["pre_processed_text"], padding='max_length', truncation=True, max_length=180)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [47]:
from datasets import Dataset
h_dataset = Dataset.from_pandas(data_df)

In [48]:
h_dataset

Dataset({
    features: ['song_id', 'song_name', 'Artist', 'Lyrics', 'Type', 'token_list', 'token_list_after_sw', 'pre_processed_text', 'word_count', 'final_data', 'label'],
    num_rows: 162
})

In [49]:
tokenized_dataset = h_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

Dataset({
    features: ['song_id', 'song_name', 'Artist', 'Lyrics', 'Type', 'token_list', 'token_list_after_sw', 'pre_processed_text', 'word_count', 'final_data', 'label', 'input_ids', 'attention_mask'],
    num_rows: 162
})

In [50]:
label_names = ["Calm", "Happy", "Sad"]
id2label = {idx:label for idx, label in enumerate(label_names)}
id2label

{0: 'Calm', 1: 'Happy', 2: 'Sad'}

In [51]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'Calm': 0, 'Happy': 1, 'Sad': 2}

In [52]:
#train, test = train_test_split(tokenized_dataset ,test_size=0.2, random_state=42)

In [53]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels, label2id=label2id, id2label=id2label)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
dataset=tokenized_dataset.shuffle().train_test_split(test_size=0.3)

In [55]:
# from transformers import TrainingArguments

# args = TrainingArguments(
#     output_dir='./results',  # output directory
#     evaluation_strategy="epoch",
#     num_train_epochs=20,  # total # of training epochs
# )

In [56]:
# import numpy as np
# from sklearn.metrics import mean_absolute_error

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     print(labels)
#     print(predictions)
#     return {"MAE": mean_absolute_error(labels, predictions)}

In [57]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

# trainer = Trainer(
#   model=model,
#   args=training_args,
#   train_dataset=tokenized_dataset["train"],
#   eval_dataset=tokenized_dataset["test"],
#   tokenizer=tokenizer,
#   compute_metrics=compute_metrics
# )

In [58]:
#trainer.train()

In [59]:
 model_checkpoint = "xlm-roberta-base"

In [60]:
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_true, y_pred)
# print(f"Accuracy: {accuracy * 100:.2f}%")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
   # print(predictions)
    predictions = np.argmax(predictions, axis=1)
    print(labels)
    print(predictions)
    print("=============================")
    return {"Accuracy": accuracy_score(labels, predictions) }

In [61]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]
batch_size = 10
num_train_epochs = 20
logging_steps = len(dataset["train"]) // (batch_size * num_train_epochs)

args = TrainingArguments(
    output_dir=f"{model_name}-sinhala-song-cls",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    #logging_steps=logging_steps,
    push_to_hub=False,
    load_best_model_at_end= True,
)

In [62]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [63]:
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.146794,0.183673
2,No log,1.147166,0.183673
3,No log,1.142666,0.183673
4,No log,1.213367,0.183673
5,No log,1.230511,0.224490
6,No log,1.105449,0.387755
7,No log,1.370530,0.204082
8,No log,1.083713,0.387755
9,No log,1.345640,0.265306
10,No log,1.160713,0.367347


[1 2 2 0 2 2 0 2 0 0 0 2 0 0 0 1 1 0 0 2 1 2 2 0 2 2 0 2 2 1 2 1 0 0 2 1 1
 2 0 0 0 2 0 0 0 1 0 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 2 0 2 2 0 2 0 0 0 2 0 0 0 1 1 0 0 2 1 2 2 0 2 2 0 2 2 1 2 1 0 0 2 1 1
 2 0 0 0 2 0 0 0 1 0 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 2 0 2 2 0 2 0 0 0 2 0 0 0 1 1 0 0 2 1 2 2 0 2 2 0 2 2 1 2 1 0 0 2 1 1
 2 0 0 0 2 0 0 0 1 0 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 2 0 2 2 0 2 0 0 0 2 0 0 0 1 1 0 0 2 1 2 2 0 2 2 0 2 2 1 2 1 0 0 2 1 1
 2 0 0 0 2 0 0 0 1 0 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 2 0 2 2 0 2 0 0 0 2 0 0 0 1 1 0 0 2 1 2 2 0 2 2 0 2 2 1 2 1 0 0 2 1 1
 2 0 0 0 2 0 0 0 1 0 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 1 1 1 2 1 1 1

TrainOutput(global_step=240, training_loss=0.7831277211507162, metrics={'train_runtime': 249.5556, 'train_samples_per_second': 9.056, 'train_steps_per_second': 0.962, 'total_flos': 209051832679200.0, 'train_loss': 0.7831277211507162, 'epoch': 20.0})

In [64]:
#tokenizer = AutoTokenizer.from_pretrained(save_directory)
#model = AutoModelForSequenceClassification.from_pretrained("./xlm-roberta-base/finetuned-marc-en", local_files_only=True)

In [65]:
#rm -rf xlm-roberta-base-sinhala-song-cls/